In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_menber(url,headers):
    r = requests.get(url, headers=headers)
    list_members = []
    mypage=BeautifulSoup(r.text,"html.parser")
    all_div = [i.div for i in mypage.find_all('article')]
    for i in all_div:
        member=[]
        member.append(i.header.h2.text.strip())
        member.append(i.header.h5.text.strip().replace('Founded',''))
        try:
            member.append(i.footer.h6.text.strip().replace('Website: ',''))
        except AttributeError:
            member.append('')
        for m in ['icon-twitter','icon-facebook','icon-gplus','icon-youtube','icon-rss']:
            try:
                member.append(i.footer.ul.find('i', attrs = {'class':m}).parent['href'])
            except AttributeError:
                member.append('')
        member.append(i.div.text.strip())
        list_members.append(member)
    return list_members

headers = {'user-agent': 'my-app/0.0.1'}
basic_url = 'https://gijn.org/member/page/'
starting_index = 1
step = 1
page_index = starting_index
all_members = []
while True:
    url = '{}{}'.format(basic_url,page_index)
    try:
        members = scrape_menber(url,headers)
    except AttributeError:
        members = []
    all_members.extend(members)
    page_index += step
    if members == []:
        break
header = ['title','founded','website','twitter','facebook','gplus','youtube','rss','description']
df = pd.DataFrame(all_members, columns = header)
df.to_csv('members.csv',na_rep='NaN',index=False)

In [28]:
df_cleaned = df
df_cleaned['country'] = [i.split('.')[-1].split('/')[0] for i in df['website']]
df_cleaned['country']

0         ru
1        org
2        com
3        org
4         by
5         id
6         za
7        net
8        org
9       info
10        no
11        eu
12        hu
13        hu
14       com
15        lv
16       com
17        bt
18        ba
19        br
20       org
21        ca
22        me
23        mk
24        np
25          
26       org
27       org
28        in
29        si
       ...  
142       ru
143       br
144       ch
145       ro
146       md
147      com
148      org
149      php
150      edu
151      org
152       rs
153     info
154       hu
155      edu
156      org
157       tz
158      org
159      org
160      com
161      org
162      org
163      org
164    se%2F
165      org
166      org
167      org
168      org
169      org
170     html
171       md
Name: country, Length: 172, dtype: object